# Covariance matrix

> A serious attempt at all the assignments is mandatory to grant access to the final exam. Refer to the course manual for more details (section *Overview* on Brightspace). 

> Please add today's topic to your knowledge graph.

**Learning goals:**
- Refresh background knowledge of covariance matrices;
- Be able to compute covariance matrices;
- Understand the eigen decomposition of covariance matrices and how eigenvalues and eigenvectors relate to the underlying data distribution;
- Understand the limitations of estimating covariance matrices from empirical data;
- Understand the solutions to the limitations.

**Solutions file:**

For this assignment, a solutions file will be released after the deadline.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.covariance import shrunk_covariance

## Exercise 1: inspect 2D data
Below we provide a function `generate_data()` to generate data from a multivariate normal distribution given a mean and covariance matrix. 
1. Inspect how we create this data;
1. Visualize the data by plotting a scatter plot (`plt.scatter()`);
1. Visualize the data by plotting two histograms (`plt.hist()` in separate plots or using `plt.subplots()`);
1. Play around by changing the properties of the data (i.e., its mean and covariance). Inspect how the data change when changing the means, and inspect how the data change when changing the entries in the covariance matrix.
1. To brush up your knowledge about covariance matrices, you might want to watch [this](https://www.youtube.com/watch?v=G16c2ZODcg8) video. On the difference between covariance and correlation, [this](https://www.youtube.com/watch?v=KDw3hC2YNFc) is a good resource. 

In [ ]:
def generate_data(N_samples=300, cov=[[2, 1], [1, 2]], mean=[0, 5]):
    X = np.random.multivariate_normal(mean, cov, N_samples)
    return X

In [ ]:
np.random.seed(42) # ensures replicable results
X = generate_data()

In [ ]:
# Answer here

## Exercise 2: covariance estimator for n-dimensional data
Below we provide a function `generate_covariance()` that samples a covariance matrix. Then, a resulting covariance matrix is fed to `generate_data()` to generate data.
1. Implement a function `estimate_covariance(X)` that calculates the covariance matrix for a given data set $X$ of size $(N_{samples} \times D_{features})$. You can test your function with the data set returned from the function `generate_data()`;
1. Compare your resulting covariance matrix with the corresponding built-in-function in the "Numpy" package (`np.cov()`) and the ground truth covariance. Please carefully check the documentation of the numpy function for the definition of the input;
1. Visualize the three covariance matrices e.g. with the function `plt.imshow()` from the "Matplotlib" package, using separate subplots with `plt.subplots()`. 

**Hint**: if you're running into trouble with the implementation of covariance estimation, inspect the shape of your output using `var.shape` and compare it with your expectations for a desired input of $N_{samples} \times  D_{features}$.

In [ ]:
def generate_covariance(D_features):
    tmp = np.random.randn(D_features, D_features)
    cov = np.dot(tmp, tmp.transpose())
    return cov

In [ ]:
np.random.seed(42) # ensures replicable results
N_samples = 200
D_features = 100
cov_true = generate_covariance(D_features)
X = generate_data(N_samples, cov_true, np.zeros(D_features))

In [ ]:
# Answer here

## (Optional) Exercise 3: eigenvectors of the covariance matrix
In this (optional) exercise, we want to shed light on the relationship between the eigenvalues / eigenvectors of the covariance matrix and the shape of the normal distribution in a 2D scenario. 
1. Visualize a randomly sampled normal distribution (e.g. by using the seaborn function `fig = sns.jointplot()`); 
1. Calculate eigenvalues and eigenvectors of the true covariance matrix using `np.linalg.eig()`;
1. Draw the estimated eigenvectors multiplied by the square root of the corresponding eigenvalues in the same plot. For this you could use seaborn's `annotate()` function; use something along the lines of this to add the arrows to your plot:
```python 
                fig.ax_joint.annotate('', xytext=(0, 0), xy=eigvects[:, i] * np.sqrt(eigvals[i]), arrowprops=dict(facecolor='red', width=2.0))
```
1. What do you observe regarding the size of the eigenvalues and the shape of the eigenvectors (i.e., refering to the covariance matrix)?
1. If the eigenvalues are over (or underestimated), what does this mean for the estimated shape of the normal distribution (i.e., its covraince matrix)?

In [ ]:
np.random.seed(3) # ensures replicable results
D_features = 2
N_samples = 500
cov_true = generate_covariance(D_features)
X = generate_data(N_samples, cov_true, np.zeros(D_features))

In [ ]:
# Answer here

## Exercise 4: eigenvalue spectrum for various sample sizes
For a fixed number of features $D_{features} = 100$, vary the number of samples $N_{samples}$ and implement the following steps:
 1. Estimate the covariance matrix from the generated samples X;
 2. Calculate the eigenvalues of the covariance matrix;
 3. Sort the eigenvalues in descending order;
 4. Plot the eigenvalues as a function of their order/rank (called eigenvalue spectrum).

This procedure should be repeated for the given sample sizes $N_{samples}=[20,50,100,300,1000]$ and all eigenvalue spectra shown in one figure. For comparison, repeat the steps (2)-(4) also for the true covariance matrix and add this eigenvalue spectrum to the created figure. 

1. What do you observe for the largest and smallest estimates of the eigenvalues?
1. What do you observe for an increasing number of N_samples?
1. Which condition for the N_samples in relation to D_features is necessary to allow an estimation of covariance?
1. How can you manipulate your data to avoid this problem, in the case of N_samples < D_features?

In [ ]:
np.random.seed(42) # ensures replicable results
D_features = 100
N_samples = [20, 50, 100, 300, 1000]
cov_true = generate_covariance(D_features)

In [ ]:
# Answer here

## Exercise 5 :  Improving covariance estimation by means of regularization
You may have observed a systematic error in the eigenvalue estimates, even for $N_{samples} > D_{features}$. Here we investigate a way to compensate for this systematic error such that the estimated spectrum approximates the true spectrum more closely. For that we use regularized estimation of the covariance matrix using a schrinkage parameter. 
1. Look-up what this shrinkage of the covariance matrix really means, what does it do? Read for instance the sklearn documentation of the function [here](https://scikit-learn.org/stable/modules/covariance.html#shrunk-covariance). We have also added an short summry below.

### Shrinkage regularization explained
Note: $N$ is used as the number of samples in a dataset and $D$ is the number of features.
#### Goal
The goal is to find a better estimate of the true covariance matrix. 

**Why**? With few samples and/or large dimensionality (especially when $N$ < $D$), large eigenvalues are overestimated and small eigenvalues are underestimated (see previous exercise). 

**How** do we combat this? Squeeze the 'pointy' estimated covariance matrix towards the (scaled) identity matrix (circle in 2D). The amount of regularization (amount of squeezing) is regulated with a scalar parameter $\gamma$ ∈ [0,1], where 0 is no regularization and 1 means just taking the (scaled) identity matrix. Intuition is shown in the figure below [1].

![Shrinkage regularization](data/shrinkage_regularization_covariance_matrix.jpg)

Figure: Left: Eigenvalue spectrum of a given covariance matrix (bold line) and eigenvalue spectra of covariance matrices estimated from a finite number of samples drawn ($N= 50,100, 200, 500$) from a corresponding Gaussian distribution. Middle: Data points drawn from a Gaussian distribution (gray dots; $D = 200$ dimensions, two dimensions selected for visualization) with true covariance matrix indicated by an orange colored ellipsoid, and estimated covariance matrix in
cyan. Right: An approximation of the true covariance matrix can be obtained as a linear interpolation between the empirical covariance matrix and a sphere of appropriate size [1].

##### Formula
$\begin{equation}
\mathbf{\tilde{\Sigma}}(\gamma):=(1-\gamma) \mathbf{\hat{\Sigma}}+\gamma \nu \mathbf{I}
\end{equation}$ 
Where, $\mathbf{\tilde{\Sigma}}$ ($D, D$) is the new regularized/shrunk covariance matrix of the data, $\gamma$ (scalar) (0 ≤ $\gamma$ ≤ 1) is
the shrinkage parameter, $\mathbf{\hat{\Sigma}}$ ($D, D$) is the estimated covariance matrix of the data, $\nu$ (scalar) is the average eigenvalue [trace](https://en.wikipedia.org/wiki/Trace_(linear_algebra)) of estimated/empirical covariance matrix $\mathbf{\hat{\Sigma}}$, and $\mathbf{I}$ is the identity matrix.

[1] *Blankertz, B., Lemm, S., Treder, M., Haufe, S., & Müller, K. R. (2011). Single-trial analysis and classification of ERP components—a tutorial. NeuroImage, 56(2), 814-825*.

### Exercises
To work with shrinkage regularization we can use the `shrunk_covariance()` function from the "sklearn" library. We have implemented a function below that uses that function. Your task now is to:

2. Estimate the empirical covariance matrix with and without regularization;
3. For the *true*, the *empirical*, and the *regularized* covariance matrices, compute the eigenvalue spectrum;
4. Plot the eigenvalue spectrums in a single figure;
5. Compute the mean squared error between the true and empirical, and the true and regularized covariance matrix (e.g., by means of the eigenvalue spectrum);
6. Report your observations.

In [ ]:
def estimate_regularized_covariance(X, shrink_param=0.3):
    cov = np.cov(X.T)
    regularized_cov = shrunk_covariance(cov, shrink_param)
    return regularized_cov

In [ ]:
np.random.seed(42) # ensures replicable results
N_samples = 120
cov_true = generate_covariance(D_features)
X = generate_data(N_samples, cov_true, np.zeros(D_features))

In [ ]:
# Answer here

## Exercise 0: Who did what?
Please provide a short description on who contributed what to your submission.

> Answer here